In [1]:
# dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin, MetaData, Table


In [2]:
# create engine to hawaii.sqlite
connect_string = "sqlite:///static/data/climateDB.db"

# reflect the tables
engine = create_engine(connect_string) 

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect =True)


In [3]:
Base.classes.keys()

['CO2_emission', 'temp_change']

In [4]:
# Save references to each table
Emission = Base.classes.CO2_emission
Temp_change = Base.classes.temp_change

In [5]:
# Create a session (link) from Python to the sqlite DB
session = Session(bind = engine)

#Filter the data for the year >= 1961
results_emission = session.query(Emission).filter(Emission.Year >= 1961)
#print(results) 

In [6]:
emission_df = pd.read_sql(results_emission.statement, session.connection())

print(emission_df.head())
results_temp = session.query(Temp_change)

#print(results) 

temp_df = pd.read_sql(results_temp.statement, session.connection())

selection = ['DecJanFeb', 'MarAprMay', 'JunJulAug', 'SepOctNov']

        Entity Code  Year  AnnualCO2emissions
0  Afghanistan  AFG  1961              490798
1  Afghanistan  AFG  1962              688594
2  Afghanistan  AFG  1963              706736
3  Afghanistan  AFG  1964              838551
4  Afghanistan  AFG  1965             1006917


In [7]:
season_df = temp_df.loc[temp_df["Months"].isin(selection)]

#filter by months
month_df = temp_df.loc[(~temp_df["Months"].isin(selection)) & (temp_df["Months"] != 'Meteorological year') ]

# filter Meteorological year
meteor_df = temp_df.loc[temp_df["Months"] == 'Meteorological year' ]
meteor_df_new = meteor_df.copy()
# Calculate avg temp per Meteorological year
meteor_df_new['avg_temp']= round(meteor_df_new.mean(axis =1),3)
meteor_id_df = meteor_df_new.set_index('Area')

session.close() 
meteor_id_df


,field1,Months,Element,Unit,1961,1962,1963,1964,1965,1966,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,avg_temp
Area,,,,,,,,,,,,,,,,,,,,,
Afghanistan,32,Meteorological year,Temperature change,°C,-0.080,-0.120,0.882,-0.727,-0.200,0.176,...,1.342,0.102,1.237,0.474,1.096,1.473,1.483,1.524,0.893,0.958
Albania,66,Meteorological year,Temperature change,°C,0.631,0.339,0.084,-0.171,-0.392,0.551,...,1.123,1.603,1.495,1.399,1.725,1.697,1.337,2.232,1.911,1.577
Algeria,100,Meteorological year,Temperature change,°C,0.186,0.101,0.045,0.212,-0.135,0.404,...,1.492,1.238,1.267,1.774,1.224,1.866,1.630,1.326,1.224,2.366
Andorra,168,Meteorological year,Temperature change,°C,0.749,0.110,-0.755,0.305,-0.500,0.403,...,1.643,1.233,0.794,1.899,1.619,1.917,1.853,1.826,1.871,3.480
Angola,202,Meteorological year,Temperature change,°C,0.053,-0.139,-0.176,-0.215,-0.181,0.193,...,0.708,0.401,0.910,0.726,1.260,1.534,0.808,1.264,1.694,3.773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Low Income Food Deficit Countries,9518,Meteorological year,Temperature change,°C,-0.085,-0.186,0.072,-0.207,-0.224,0.080,...,0.872,0.618,0.875,0.820,1.061,1.292,1.276,1.094,1.244,159.063
Net Food Importing Developing Countries,9552,Meteorological year,Temperature change,°C,-0.015,-0.096,0.116,-0.204,-0.155,0.129,...,0.878,0.692,0.977,0.971,1.269,1.374,1.292,1.222,1.412,159.672
Annex I countries,9586,Meteorological year,Temperature change,°C,0.453,0.161,0.172,-0.366,-0.409,0.270,...,1.090,1.512,1.109,1.119,1.616,2.098,1.607,1.397,1.627,160.359


In [8]:
#calculate overall avg co2 emission per country
avg_co2 =  emission_df.groupby("Entity").agg({'AnnualCO2emissions':'mean'})
avg_co2 = round(avg_co2/1000000,3) ## converting GT to Mega ton for the tooltip

In [13]:
#Merge Temp_change by meteor year per country to Avg_Co2 Emission df
merged_co2_country =meteor_id_df.merge(avg_co2, how = 'left',  left_index=True, right_index=True,)
merged_co2_country

,field1,Months,Element,Unit,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,avg_temp,AnnualCO2emissions
Area,,,,,,,,,,,,,,,,,,,,,
Afghanistan,32,Meteorological year,Temperature change,°C,-0.080,-0.120,0.882,-0.727,-0.200,0.176,...,0.102,1.237,0.474,1.096,1.473,1.483,1.524,0.893,0.958,3.036
Albania,66,Meteorological year,Temperature change,°C,0.631,0.339,0.084,-0.171,-0.392,0.551,...,1.603,1.495,1.399,1.725,1.697,1.337,2.232,1.911,1.577,4.503
Algeria,100,Meteorological year,Temperature change,°C,0.186,0.101,0.045,0.212,-0.135,0.404,...,1.238,1.267,1.774,1.224,1.866,1.630,1.326,1.224,2.366,74.340
Andorra,168,Meteorological year,Temperature change,°C,0.749,0.110,-0.755,0.305,-0.500,0.403,...,1.233,0.794,1.899,1.619,1.917,1.853,1.826,1.871,3.480,0.246
Angola,202,Meteorological year,Temperature change,°C,0.053,-0.139,-0.176,-0.215,-0.181,0.193,...,0.401,0.910,0.726,1.260,1.534,0.808,1.264,1.694,3.773,11.971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Low Income Food Deficit Countries,9518,Meteorological year,Temperature change,°C,-0.085,-0.186,0.072,-0.207,-0.224,0.080,...,0.618,0.875,0.820,1.061,1.292,1.276,1.094,1.244,159.063,NaN
Net Food Importing Developing Countries,9552,Meteorological year,Temperature change,°C,-0.015,-0.096,0.116,-0.204,-0.155,0.129,...,0.692,0.977,0.971,1.269,1.374,1.292,1.222,1.412,159.672,NaN
Annex I countries,9586,Meteorological year,Temperature change,°C,0.453,0.161,0.172,-0.366,-0.409,0.270,...,1.512,1.109,1.119,1.616,2.098,1.607,1.397,1.627,160.359,NaN


In [14]:

#find null
merged_co2_country.isna().sum()
#fill 0
merged_co2_country = merged_co2_country.fillna(0)
#find null again
merged_co2_country.isna().sum()



field1                0
Months                0
Element               0
Unit                  0
1961                  0
                     ..
2017                  0
2018                  0
2019                  0
avg_temp              0
AnnualCO2emissions    0
Length: 65, dtype: int64

In [16]:
#Get New Countries from the merged DF
New_Countires = merged_co2_country.index

#Create a dictionary holding above values
 #meta ={
#    'country' : country_name,
#    'demo_info' : [web scraped data],
#    'tool_tip' : [{'c_name': country_name,
#                'avg_temp':avg_temp,
#                'avg_co2' : new_Avg_c02,
#                'population':population from web scraping
#                }]
#   } 

#New Code---------------------------------------------------

meta = []

for country in New_Countires:
    temp_co2_obj = {
                    "Country":country,
                    "Avg Temp Change":merged_co2_country.loc[country,"avg_temp"],
                    "Avg Co2 Change":merged_co2_country.loc[country,"AnnualCO2emissions"],
                    }
    meta.append(temp_co2_obj)




meta

[{'Country': 'Afghanistan', 'Avg Temp Change': 0.958, 'Avg Co2 Change': 3.036},
 {'Country': 'Albania', 'Avg Temp Change': 1.577, 'Avg Co2 Change': 4.503},
 {'Country': 'Algeria', 'Avg Temp Change': 2.366, 'Avg Co2 Change': 74.34},
 {'Country': 'Andorra', 'Avg Temp Change': 3.48, 'Avg Co2 Change': 0.246},
 {'Country': 'Angola', 'Avg Temp Change': 3.773, 'Avg Co2 Change': 11.971},
 {'Country': 'Anguilla', 'Avg Temp Change': 4.185, 'Avg Co2 Change': 0.056},
 {'Country': 'Antarctica', 'Avg Temp Change': 4.655, 'Avg Co2 Change': 0.0},
 {'Country': 'Argentina', 'Avg Temp Change': 5.894, 'Avg Co2 Change': 124.014},
 {'Country': 'Australia', 'Avg Temp Change': 7.76, 'Avg Co2 Change': 273.351},
 {'Country': 'Austria', 'Avg Temp Change': 8.636, 'Avg Co2 Change': 59.374},
 {'Country': 'Bahamas', 'Avg Temp Change': 9.45, 'Avg Co2 Change': 2.751},
 {'Country': 'Bahrain', 'Avg Temp Change': 10.288, 'Avg Co2 Change': 14.123},
 {'Country': 'Bangladesh',
  'Avg Temp Change': 10.391,
  'Avg Co2 Change'